In [1]:
from scipy import stats
import pickle as pkl
from scipy.stats import spearmanr

In [2]:
from sklearn.utils import resample

In [3]:
import numpy as np
from statistics import mean, median

In [4]:
def ranksumdiff(a,b):
	totalpoints = []
	for i in a:
		totalpoints.append(i)
	for i in b:
		totalpoints.append(i)

	totalpoints.sort()
	rank_points = {}
	c = 1
	for i in totalpoints:
		if(i not in rank_points):
			rank_points[i] = []
		rank_points[i].append(c)
		c+=1
	rank_value = {}
	for i in rank_points:
		rank_value[i] = mean(rank_points[i])
	ars = []
	brs = []

	for i in a:
		ars.append(rank_value[i])

	for i in b:
		brs.append(rank_value[i])

	return mean(ars) - mean(brs)

def bootstrap(a, b, u0, iterations):
	n_iter = iterations
	m = len(a)
	n = len(b)
	centreu = (m*n)/2
	totallist = []
	for i in a:
		totallist.append(i)
	for i in b:
		totallist.append(i)
	count = 0
	totallist = np.asarray(totallist)
	for i in range(n_iter):
		sample = np.random.choice(totallist, m+n, replace=True)
		samplea = sample[:m]
		sampleb = sample[m:]
		u,p = stats.mannwhitneyu(samplea, sampleb, alternative='two-sided')
		if(abs(u-centreu)>=abs(u0-centreu)):
			count+=1
	return count/n_iter

def mannwhitneyu_bootstrap(low, high, iterations):
	u,p = stats.mannwhitneyu(low, high, alternative='two-sided')
	if ranksumdiff(low, high)>0:
		dom = "Low"
	else:
		dom = "High"
	bpval = ''
	if(p<0.05):
		bpval = bootstrap(low, high, u, iterations)
	return str(u)+", "+str(p)+"("+dom+", "+str(bpval)+")"

In [5]:
def Get_Users_topandbottom(dataset, pers, num, emotion):
    distril = []
    distrih = [] 

    temp1 = {}
    temp2 = {}
    for i in dataset:
        temp1[i] = dataset[i][pers]
        temp2[i] = dataset[i][pers]

    temp1 = dict(sorted(temp1.items(), key=lambda item: item[1], reverse=False))
    temp2 = dict(sorted(temp2.items(), key=lambda item: item[1], reverse=True))

    t = 0
    b = 0
    for i in temp1:
        if dataset[i]['Ext_song_plays'] >= th_songs and dataset[i]['Ext_song_plays']/dataset[i]['Total_song_plays'] >= th_hitrate:
            distril.append(dataset[i][emotion])
        t += 1

        if t == num:
            break
    
    for i in temp2:
        if dataset[i]['Ext_song_plays'] >= th_songs and dataset[i]['Ext_song_plays']/dataset[i]['Total_song_plays'] >= th_hitrate:
            distrih.append(dataset[i][emotion])
        b += 1

        if b == num:
            break
    return distril, distrih


In [6]:
def Get_Users_LH(pers, med, dataset, emotion):
    distril = []
    distrih = [] 

    for i in range(len(dataset)):
        if dataset[i][pers] <= median(med) - th_median and dataset[i]['Ext_song_plays'] >= th_songs\
        and dataset[i]['Ext_song_plays']/dataset[i]['Total_song_plays'] >= th_hitrate:
            distril.append(dataset[i][emotion])
            
        if dataset[i][pers] >= median(med) + th_median and dataset[i]['Ext_song_plays'] >= th_songs \
         and dataset[i]['Ext_song_plays']/dataset[i]['Total_song_plays'] >= th_hitrate:
            distrih.append(dataset[i][emotion])
    
    return distril, distrih

In [7]:
PerCorrLH = pkl.load(open('../data/percorr_lh_vectors.pkl', 'rb'))

In [8]:
PerCorrLH[0]

{'Total_song_plays': 6,
 'Total_unique_songs': 6,
 'Ext_song_plays': 6,
 'Ext_unique_songs': 6,
 'Non_Instr_song_plays': 6,
 'Non_Instr_unique_songs': 6,
 'Instr_unique_song_plays': 0,
 'Instr_unique_songs': 0,
 'freq': [1, 1, 1, 1, 1, 1],
 'Joy': -0.4227942315240701,
 'Angry': -0.2609129746754964,
 'Sad': 1.5501137971878052,
 'Relaxed': -0.31631347474952537,
 'open': 5.0,
 'con': 1.0,
 'ext': 4.75,
 'agr': 2.5,
 'neu': 4.0}

## General Parameters

In [9]:
hitrate = []
for i in range(len(PerCorrLH)):
    hitrate.append(PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'])

In [10]:
np.mean(np.array(hitrate))

0.82340182905041

In [11]:
th_median = 0.25
th_songs = 100
th_hitrate = 0.7

In [12]:
c = 0
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
            c += 1

In [13]:
c

1120

In [14]:
ope = []
for i in range(len(PerCorrLH)):
    ope.append(PerCorrLH[i]['open'])
print('Open:%s'%(median(ope)))

con = []
for i in range(len(PerCorrLH)):
    con.append(PerCorrLH[i]['con'])
print('Con:%s'%(median(con)))

ext = []
for i in range(len(PerCorrLH)):
    ext.append(PerCorrLH[i]['ext'])
print('Ext:%s'%(median(ext)))

agr = []
for i in range(len(PerCorrLH)):
    agr.append(PerCorrLH[i]['agr'])
print('Agr:%s'%(median(agr)))

neu = []
for i in range(len(PerCorrLH)):
    neu.append(PerCorrLH[i]['neu'])
print('Neu:%s'%(median(neu)))


Open:4.25
Con:3.15
Ext:3.05
Agr:3.5
Neu:2.85


# Median Split

In [15]:
distriopenl,distriopenh = Get_Users_LH('open', ope, PerCorrLH, 'Joy')
districonl,districonh = Get_Users_LH('con', con, PerCorrLH, 'Joy')
distriextl,distriexth = Get_Users_LH('ext', ext, PerCorrLH, 'Joy')
distriagrl,distriagrh = Get_Users_LH('agr', agr, PerCorrLH, 'Joy')
distrineul,distrineuh = Get_Users_LH('neu', neu, PerCorrLH, 'Joy')

print('\n----Joy----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl, distriopenh, 10000))
print("con",mannwhitneyu_bootstrap(districonl, districonh, 10000))
print("ext",mannwhitneyu_bootstrap(distriextl, distriexth, 10000))
print("agr",mannwhitneyu_bootstrap(distriagrl, distriagrh, 10000))
print("neu",mannwhitneyu_bootstrap(distrineul, distrineuh, 10000))
print('\n--------\n')

distriopenl,distriopenh = Get_Users_LH('open', ope, PerCorrLH, 'Angry')
districonl,districonh = Get_Users_LH('con', con, PerCorrLH, 'Angry')
distriextl,distriexth = Get_Users_LH('ext', ext, PerCorrLH, 'Angry')
distriagrl,distriagrh = Get_Users_LH('agr', agr, PerCorrLH, 'Angry')
distrineul,distrineuh = Get_Users_LH('neu', neu, PerCorrLH, 'Angry')

print('\n----Angry----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl, distriopenh, 10000))
print("con",mannwhitneyu_bootstrap(districonl, districonh, 10000))
print("ext",mannwhitneyu_bootstrap(distriextl, distriexth, 10000))
print("agr",mannwhitneyu_bootstrap(distriagrl, distriagrh, 10000))
print("neu",mannwhitneyu_bootstrap(distrineul, distrineuh, 10000))
print('\n--------\n')

distriopenl,distriopenh = Get_Users_LH('open', ope, PerCorrLH, 'Sad')
districonl,districonh = Get_Users_LH('con', con, PerCorrLH, 'Sad')
distriextl,distriexth = Get_Users_LH('ext', ext, PerCorrLH, 'Sad')
distriagrl,distriagrh = Get_Users_LH('agr', agr, PerCorrLH, 'Sad')
distrineul,distrineuh = Get_Users_LH('neu', neu, PerCorrLH, 'Sad')

print('\n----Sad----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl, distriopenh, 10000))
print("con",mannwhitneyu_bootstrap(districonl, districonh, 10000))
print("ext",mannwhitneyu_bootstrap(distriextl, distriexth, 10000))
print("agr",mannwhitneyu_bootstrap(distriagrl, distriagrh, 10000))
print("neu",mannwhitneyu_bootstrap(distrineul, distrineuh, 10000))
print('\n--------\n')


distriopenl,distriopenh = Get_Users_LH('open', ope, PerCorrLH, 'Relaxed')
districonl,districonh = Get_Users_LH('con', con, PerCorrLH, 'Relaxed')
distriextl,distriexth = Get_Users_LH('ext', ext, PerCorrLH, 'Relaxed')
distriagrl,distriagrh = Get_Users_LH('agr', agr, PerCorrLH, 'Relaxed')
distrineul,distrineuh = Get_Users_LH('neu', neu, PerCorrLH, 'Relaxed')

print('\n----Relaxed----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl, distriopenh, 10000))
print("con",mannwhitneyu_bootstrap(districonl, districonh, 10000))
print("ext",mannwhitneyu_bootstrap(distriextl, distriexth, 10000))
print("agr",mannwhitneyu_bootstrap(distriagrl, distriagrh, 10000))
print("neu",mannwhitneyu_bootstrap(distrineul, distrineuh, 10000))
print('\n--------\n')



----Joy----

Open 63470.0, 0.029766367895205117(High, 0.0314)
con 78644.0, 0.051409154110504514(High, )
ext 88826.0, 0.04282710215512266(High, 0.0433)
agr 72831.0, 0.03315732627340908(High, 0.0359)
neu 100007.0, 0.3126817353346276(Low, )

--------


----Angry----

Open 83028.0, 8.858997964882965e-06(Low, 0.0)
con 89776.0, 0.1957859914464808(Low, )
ext 105008.0, 0.022790936908603024(Low, 0.0233)
agr 90120.0, 0.0015217637783396943(Low, 0.0016)
neu 88429.0, 0.0380832695859816(High, 0.0363)

--------


----Sad----

Open 62339.0, 0.010593699548981771(High, 0.0101)
con 91741.0, 0.062052387786648976(Low, )
ext 91310.0, 0.17227518156875465(High, )
agr 73153.0, 0.04220990795496045(High, 0.0434)
neu 95243.0, 0.7956134358841209(High, )

--------


----Relaxed----

Open 54501.0, 1.9134089889697009e-07(High, 0.0)
con 80602.0, 0.16825980612031843(High, )
ext 92882.0, 0.3436613886557419(High, )
agr 71026.0, 0.007283328282984036(High, 0.0072)
neu 107112.0, 0.003707308424877341(Low, 0.0035)

--------


# Top And Bottom 100 Users

In [16]:
distriopenl100,distriopenh100 = Get_Users_topandbottom(PerCorrLH, 'open', 100, 'Joy')
districonl100,districonh100 = Get_Users_topandbottom(PerCorrLH, 'con', 100, 'Joy')
distriextl100,distriexth100 = Get_Users_topandbottom(PerCorrLH, 'ext', 100, 'Joy')
distriagrl100,distriagrh100 = Get_Users_topandbottom(PerCorrLH, 'agr', 100, 'Joy')
distrineul100,distrineuh100 = Get_Users_topandbottom(PerCorrLH, 'neu', 100, 'Joy')

print('\n----Joy----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl100, distriopenh100, 10000))
print("Con", mannwhitneyu_bootstrap(districonl100, districonh100, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl100, distriexth100, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl100, distriagrh100, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul100, distrineuh100, 10000))
print('\n--------\n')

distriopenl100,distriopenh100 = Get_Users_topandbottom(PerCorrLH, 'open', 100, 'Angry')
districonl100,districonh100 = Get_Users_topandbottom(PerCorrLH, 'con', 100, 'Angry')
distriextl100,distriexth100 = Get_Users_topandbottom(PerCorrLH, 'ext', 100, 'Angry')
distriagrl100,distriagrh100 = Get_Users_topandbottom(PerCorrLH, 'agr', 100, 'Angry')
distrineul100,distrineuh100 = Get_Users_topandbottom(PerCorrLH, 'neu', 100, 'Angry')

print('\n----Angry----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl100, distriopenh100, 10000))
print("Con", mannwhitneyu_bootstrap(districonl100, districonh100, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl100, distriexth100, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl100, distriagrh100, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul100, distrineuh100, 10000))
print('\n--------\n')

distriopenl100,distriopenh100 = Get_Users_topandbottom(PerCorrLH, 'open', 100, 'Sad')
districonl100,districonh100 = Get_Users_topandbottom(PerCorrLH, 'con', 100, 'Sad')
distriextl100,distriexth100 = Get_Users_topandbottom(PerCorrLH, 'ext', 100, 'Sad')
distriagrl100,distriagrh100 = Get_Users_topandbottom(PerCorrLH, 'agr', 100, 'Sad')
distrineul100,distrineuh100 = Get_Users_topandbottom(PerCorrLH, 'neu', 100, 'Sad')

print('\n----Sad----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl100, distriopenh100, 10000))
print("Con", mannwhitneyu_bootstrap(districonl100, districonh100, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl100, distriexth100, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl100, distriagrh100, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul100, distrineuh100, 10000))
print('\n--------\n')

distriopenl100,distriopenh100 = Get_Users_topandbottom(PerCorrLH, 'open', 100, 'Relaxed')
districonl100,districonh100 = Get_Users_topandbottom(PerCorrLH, 'con', 100, 'Relaxed')
distriextl100,distriexth100 = Get_Users_topandbottom(PerCorrLH, 'ext', 100, 'Relaxed')
distriagrl100,distriagrh100 = Get_Users_topandbottom(PerCorrLH, 'agr', 100, 'Relaxed')
distrineul100,distrineuh100 = Get_Users_topandbottom(PerCorrLH, 'neu', 100, 'Relaxed')

print('\n----Relaxed----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl100, distriopenh100, 10000))
print("Con", mannwhitneyu_bootstrap(districonl100, districonh100, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl100, distriexth100, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl100, distriagrh100, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul100, distrineuh100, 10000))
print('\n--------\n')


----Joy----

Open 2171.0, 0.031002134356004997(High, 0.032)
Con 2162.0, 0.13784579551708565(High, )
Ext 2338.0, 0.03802019397198564(High, 0.039)
Agr 2601.0, 0.5243694807485411(High, )
Neu 3660.0, 0.03850445818935276(Low, 0.0376)

--------


----Angry----

Open 3987.0, 1.5152559138423338e-06(Low, 0.0)
Con 2539.0, 0.9659980604223651(Low, )
Ext 3418.0, 0.06053480349180234(Low, )
Agr 3205.0, 0.0978791440875964(Low, )
Neu 2523.0, 0.05419578086117417(High, )

--------


----Sad----

Open 2045.0, 0.008274634027715012(High, 0.0071)
Con 3033.0, 0.04054287635434703(Low, 0.0405)
Ext 3007.0, 0.7101062334662778(Low, )
Agr 2076.0, 0.00848802129353642(High, 0.0085)
Neu 2792.0, 0.3274132601703468(High, )

--------


----Relaxed----

Open 1612.0, 1.688236269273047e-05(High, 0.0)
Con 2468.0, 0.8091220725741175(High, )
Ext 2603.0, 0.2695264913209884(High, )
Agr 2421.0, 0.18658199014806187(High, )
Neu 3672.0, 0.034717211579414056(Low, 0.0359)

--------



In [17]:
print(len(distriopenl),len(distriopenh))
print(len(distriopenl100),len(distriopenh100))

396 353
71 77


# Top and Bottom 250

In [18]:
distriopenl250,distriopenh250 = Get_Users_topandbottom(PerCorrLH, 'open', 250, 'Joy')
districonl250,districonh250 = Get_Users_topandbottom(PerCorrLH, 'con', 250, 'Joy')
distriextl250,distriexth250 = Get_Users_topandbottom(PerCorrLH, 'ext', 250, 'Joy')
distriagrl250,distriagrh250 = Get_Users_topandbottom(PerCorrLH, 'agr', 250, 'Joy')
distrineul250,distrineuh250 = Get_Users_topandbottom(PerCorrLH, 'neu', 250, 'Joy')

print('\n----Joy----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl250, distriopenh250, 10000))
print("Con", mannwhitneyu_bootstrap(districonl250, districonh250, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl250, distriexth250, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl250, distriagrh250, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul250, distrineuh250, 10000))
print('\n--------\n')

distriopenl250,distriopenh250 = Get_Users_topandbottom(PerCorrLH, 'open', 250, 'Angry')
districonl250,districonh250 = Get_Users_topandbottom(PerCorrLH, 'con', 250, 'Angry')
distriextl250,distriexth250 = Get_Users_topandbottom(PerCorrLH, 'ext', 250, 'Angry')
distriagrl250,distriagrh250 = Get_Users_topandbottom(PerCorrLH, 'agr', 250, 'Angry')
distrineul250,distrineuh250 = Get_Users_topandbottom(PerCorrLH, 'neu', 250, 'Angry')

print('\n----Angry----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl250, distriopenh250, 10000))
print("Con", mannwhitneyu_bootstrap(districonl250, districonh250, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl250, distriexth250, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl250, distriagrh250, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul250, distrineuh250, 10000))
print('\n--------\n')

distriopenl250,distriopenh250 = Get_Users_topandbottom(PerCorrLH, 'open', 250, 'Sad')
districonl250,districonh250 = Get_Users_topandbottom(PerCorrLH, 'con', 250, 'Sad')
distriextl250,distriexth250 = Get_Users_topandbottom(PerCorrLH, 'ext', 250, 'Sad')
distriagrl250,distriagrh250 = Get_Users_topandbottom(PerCorrLH, 'agr', 250, 'Sad')
distrineul250,distrineuh250 = Get_Users_topandbottom(PerCorrLH, 'neu', 250, 'Sad')

print('\n----Sad----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl250, distriopenh250, 10000))
print("Con", mannwhitneyu_bootstrap(districonl250, districonh250, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl250, distriexth250, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl250, distriagrh250, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul250, distrineuh250, 10000))
print('\n--------\n')

distriopenl250,distriopenh250 = Get_Users_topandbottom(PerCorrLH, 'open', 250, 'Relaxed')
districonl250,districonh250 = Get_Users_topandbottom(PerCorrLH, 'con', 250, 'Relaxed')
distriextl250,distriexth250 = Get_Users_topandbottom(PerCorrLH, 'ext', 250, 'Relaxed')
distriagrl250,distriagrh250 = Get_Users_topandbottom(PerCorrLH, 'agr', 250, 'Relaxed')
distrineul250,distrineuh250 = Get_Users_topandbottom(PerCorrLH, 'neu', 250, 'Relaxed')

print('\n----Relaxed----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl250, distriopenh250, 10000))
print("Con", mannwhitneyu_bootstrap(districonl250, districonh250, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl250, distriexth250, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl250, distriagrh250, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul250, distrineuh250, 10000))
print('\n--------\n')


----Joy----

Open 14101.0, 0.0012113239493009682(High, 0.0018)
Con 15377.0, 0.12047819106243586(High, )
Ext 15357.0, 0.009741483646991279(High, 0.0119)
Agr 14866.0, 0.006295214955637316(High, 0.0055)
Neu 20574.0, 0.15733534299513374(Low, )

--------


----Angry----

Open 23219.0, 4.130430878525308e-08(Low, 0.0)
Con 17944.0, 0.3409150274455489(Low, )
Ext 20557.0, 0.024223694919060967(Low, 0.0195)
Agr 21814.0, 0.00012488779613754525(Low, 0.0)
Neu 17092.0, 0.08648221693716864(High, )

--------


----Sad----

Open 15549.0, 0.06416468889513788(High, )
Con 18476.0, 0.14109149533811438(Low, )
Ext 18261.0, 0.9070215174896687(Low, )
Agr 15139.0, 0.013365614429345774(High, 0.0128)
Neu 18224.0, 0.4858435001333734(High, )

--------


----Relaxed----

Open 11552.0, 1.3902082488859885e-08(High, 0.0)
Con 16008.0, 0.348893140808583(High, )
Ext 16566.0, 0.14436692429634843(High, )
Agr 14627.0, 0.0030967533010761348(High, 0.0026)
Neu 21946.0, 0.008118662199931322(Low, 0.0079)

--------



In [19]:
print(len(distriopenl),len(distriopenh))
print(len(distriopenl250),len(distriopenh250))

396 353
187 187


# Top and Bottom 500

In [20]:
distriopenl500,distriopenh500 = Get_Users_topandbottom(PerCorrLH, 'open', 500, 'Joy')
districonl500,districonh500 = Get_Users_topandbottom(PerCorrLH, 'con', 500, 'Joy')
distriextl500,distriexth500 = Get_Users_topandbottom(PerCorrLH, 'ext', 500, 'Joy')
distriagrl500,distriagrh500 = Get_Users_topandbottom(PerCorrLH, 'agr', 500, 'Joy')
distrineul500,distrineuh500 = Get_Users_topandbottom(PerCorrLH, 'neu', 500, 'Joy')

print('\n----Joy----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl500, distriopenh500, 10000))
print("Con", mannwhitneyu_bootstrap(districonl500, districonh500, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl500, distriexth500, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl500, distriagrh500, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul500, distrineuh500, 10000))
print('\n--------\n')

distriopenl500,distriopenh500 = Get_Users_topandbottom(PerCorrLH, 'open', 500, 'Angry')
districonl500,districonh500 = Get_Users_topandbottom(PerCorrLH, 'con', 500, 'Angry')
distriextl500,distriexth500 = Get_Users_topandbottom(PerCorrLH, 'ext', 500, 'Angry')
distriagrl500,distriagrh500 = Get_Users_topandbottom(PerCorrLH, 'agr', 500, 'Angry')
distrineul500,distrineuh500 = Get_Users_topandbottom(PerCorrLH, 'neu', 500, 'Angry')

print('\n----Angry----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl500, distriopenh500, 10000))
print("Con", mannwhitneyu_bootstrap(districonl500, districonh500, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl500, distriexth500, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl500, distriagrh500, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul500, distrineuh500, 10000))
print('\n--------\n')

distriopenl500,distriopenh500 = Get_Users_topandbottom(PerCorrLH, 'open', 500, 'Sad')
districonl500,districonh500 = Get_Users_topandbottom(PerCorrLH, 'con', 500, 'Sad')
distriextl500,distriexth500 = Get_Users_topandbottom(PerCorrLH, 'ext', 500, 'Sad')
distriagrl500,distriagrh500 = Get_Users_topandbottom(PerCorrLH, 'agr', 500, 'Sad')
distrineul500,distrineuh500 = Get_Users_topandbottom(PerCorrLH, 'neu', 500, 'Sad')

print('\n----Sad----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl500, distriopenh500, 10000))
print("Con", mannwhitneyu_bootstrap(districonl500, districonh500, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl500, distriexth500, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl500, distriagrh500, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul500, distrineuh500, 10000))
print('\n--------\n')

distriopenl500,distriopenh500 = Get_Users_topandbottom(PerCorrLH, 'open', 500, 'Relaxed')
districonl500,districonh500 = Get_Users_topandbottom(PerCorrLH, 'con', 500, 'Relaxed')
distriextl500,distriexth500 = Get_Users_topandbottom(PerCorrLH, 'ext', 500, 'Relaxed')
distriagrl500,distriagrh500 = Get_Users_topandbottom(PerCorrLH, 'agr', 500, 'Relaxed')
distrineul500,distrineuh500 = Get_Users_topandbottom(PerCorrLH, 'neu', 500, 'Relaxed')

print('\n----Relaxed----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl500, distriopenh500, 10000))
print("Con", mannwhitneyu_bootstrap(districonl500, districonh500, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl500, distriexth500, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl500, distriagrh500, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul500, distrineuh500, 10000))
print('\n--------\n')


----Joy----

Open 64644.0, 0.03679676887803687(High, 0.0359)
Con 66039.0, 0.09579702969208376(High, )
Ext 67479.0, 0.05746851072770415(High, )
Agr 66661.0, 0.0588523149890786(High, )
Neu 73631.0, 0.5264464331032773(Low, )

--------


----Angry----

Open 84355.0, 6.293217432796285e-06(Low, 0.0)
Con 74634.0, 0.22732658675053807(Low, )
Ext 79835.0, 0.03263183152575448(Low, 0.0328)
Agr 81607.0, 0.0023684607497823722(Low, 0.0017)
Neu 66284.0, 0.07101679628330904(High, )

--------


----Sad----

Open 62724.0, 0.006307480791099843(High, 0.0058)
Con 76622.0, 0.0612294559997131(Low, )
Ext 69954.0, 0.27520217621143983(High, )
Agr 66227.0, 0.04211005519704534(High, 0.0454)
Neu 72404.0, 0.8211343430515472(Low, )

--------


----Relaxed----

Open 55289.0, 1.7640310305029398e-07(High, 0.0)
Con 66359.0, 0.11908899954708965(High, )
Ext 70862.0, 0.4268917201064859(High, )
Agr 64353.0, 0.008037564358507986(High, 0.0084)
Neu 78837.0, 0.018187045443337095(Low, 0.0164)

--------



# Top 500 songs

In [21]:
PerCorrLH_top500 = pkl.load(open('../data/percorr_lhtop500_vectors.pkl', 'rb'))

# Median Split

In [22]:
distriopenl,distriopenh = Get_Users_LH('open', ope, PerCorrLH_top500, 'Joy')
districonl,districonh = Get_Users_LH('con', con, PerCorrLH_top500, 'Joy')
distriextl,distriexth = Get_Users_LH('ext', ext, PerCorrLH_top500, 'Joy')
distriagrl,distriagrh = Get_Users_LH('agr', agr, PerCorrLH_top500, 'Joy')
distrineul,distrineuh = Get_Users_LH('neu', neu, PerCorrLH_top500, 'Joy')

print('\n----Joy----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl, distriopenh, 10000))
print("con",mannwhitneyu_bootstrap(districonl, districonh, 10000))
print("ext",mannwhitneyu_bootstrap(distriextl, distriexth, 10000))
print("agr",mannwhitneyu_bootstrap(distriagrl, distriagrh, 10000))
print("neu",mannwhitneyu_bootstrap(distrineul, distrineuh, 10000))
print('\n--------\n')

distriopenl,distriopenh = Get_Users_LH('open', ope, PerCorrLH_top500, 'Angry')
districonl,districonh = Get_Users_LH('con', con, PerCorrLH_top500, 'Angry')
distriextl,distriexth = Get_Users_LH('ext', ext, PerCorrLH_top500, 'Angry')
distriagrl,distriagrh = Get_Users_LH('agr', agr, PerCorrLH_top500, 'Angry')
distrineul,distrineuh = Get_Users_LH('neu', neu, PerCorrLH_top500, 'Angry')

print('\n----Angry----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl, distriopenh, 10000))
print("con",mannwhitneyu_bootstrap(districonl, districonh, 10000))
print("ext",mannwhitneyu_bootstrap(distriextl, distriexth, 10000))
print("agr",mannwhitneyu_bootstrap(distriagrl, distriagrh, 10000))
print("neu",mannwhitneyu_bootstrap(distrineul, distrineuh, 10000))
print('\n--------\n')

distriopenl,distriopenh = Get_Users_LH('open', ope, PerCorrLH_top500, 'Sad')
districonl,districonh = Get_Users_LH('con', con, PerCorrLH_top500, 'Sad')
distriextl,distriexth = Get_Users_LH('ext', ext, PerCorrLH_top500, 'Sad')
distriagrl,distriagrh = Get_Users_LH('agr', agr, PerCorrLH_top500, 'Sad')
distrineul,distrineuh = Get_Users_LH('neu', neu, PerCorrLH_top500, 'Sad')

print('\n----Sad----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl, distriopenh, 10000))
print("con",mannwhitneyu_bootstrap(districonl, districonh, 10000))
print("ext",mannwhitneyu_bootstrap(distriextl, distriexth, 10000))
print("agr",mannwhitneyu_bootstrap(distriagrl, distriagrh, 10000))
print("neu",mannwhitneyu_bootstrap(distrineul, distrineuh, 10000))
print('\n--------\n')


distriopenl,distriopenh = Get_Users_LH('open', ope, PerCorrLH_top500, 'Relaxed')
districonl,districonh = Get_Users_LH('con', con, PerCorrLH_top500, 'Relaxed')
distriextl,distriexth = Get_Users_LH('ext', ext, PerCorrLH_top500, 'Relaxed')
distriagrl,distriagrh = Get_Users_LH('agr', agr, PerCorrLH_top500, 'Relaxed')
distrineul,distrineuh = Get_Users_LH('neu', neu, PerCorrLH_top500, 'Relaxed')

print('\n----Relaxed----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl, distriopenh, 10000))
print("con",mannwhitneyu_bootstrap(districonl, districonh, 10000))
print("ext",mannwhitneyu_bootstrap(distriextl, distriexth, 10000))
print("agr",mannwhitneyu_bootstrap(distriagrl, distriagrh, 10000))
print("neu",mannwhitneyu_bootstrap(distrineul, distrineuh, 10000))
print('\n--------\n')



----Joy----

Open 60133.0, 0.027768585991390517(High, 0.0311)
con 76178.0, 0.023626088642868335(High, 0.0233)
ext 83574.0, 0.0179102584870361(High, 0.0175)
agr 70566.5, 0.02366103222093275(High, 0.022)
neu 97386.0, 0.15973356428314844(Low, )

--------


----Angry----

Open 79897.0, 2.051399961652492e-06(Low, 0.0)
con 87261.0, 0.314150186475903(Low, )
ext 101179.0, 0.013301740179821359(Low, 0.0137)
agr 86575.5, 0.006167769739787389(Low, 0.007)
neu 84444.0, 0.031519651505234836(High, 0.0318)

--------


----Sad----

Open 59889.0, 0.022234803076315434(High, 0.0218)
con 93939.0, 0.0029120389356081097(Low, 0.0033)
ext 89067.0, 0.3918411685853529(High, )
agr 76907.5, 0.7783301307068564(High, )
neu 90026.0, 0.5376196809245448(High, )

--------


----Relaxed----

Open 52395.0, 8.597783786885858e-07(High, 0.0)
con 78260.0, 0.09917457255581467(High, )
ext 87375.0, 0.18624439123942904(High, )
agr 67672.5, 0.0015415282636072124(High, 0.0019)
neu 101059.0, 0.01571904962542788(Low, 0.0172)

-------

# Top and bottom 100

In [23]:
distriopenl100,distriopenh100 = Get_Users_topandbottom(PerCorrLH_top500, 'open', 100, 'Joy')
districonl100,districonh100 = Get_Users_topandbottom(PerCorrLH_top500, 'con', 100, 'Joy')
distriextl100,distriexth100 = Get_Users_topandbottom(PerCorrLH_top500, 'ext', 100, 'Joy')
distriagrl100,distriagrh100 = Get_Users_topandbottom(PerCorrLH_top500, 'agr', 100, 'Joy')
distrineul100,distrineuh100 = Get_Users_topandbottom(PerCorrLH_top500, 'neu', 100, 'Joy')

print('\n----Joy----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl100, distriopenh100, 10000))
print("Con", mannwhitneyu_bootstrap(districonl100, districonh100, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl100, distriexth100, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl100, distriagrh100, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul100, distrineuh100, 10000))
print('\n--------\n')

distriopenl100,distriopenh100 = Get_Users_topandbottom(PerCorrLH_top500, 'open', 100, 'Angry')
districonl100,districonh100 = Get_Users_topandbottom(PerCorrLH_top500, 'con', 100, 'Angry')
distriextl100,distriexth100 = Get_Users_topandbottom(PerCorrLH_top500, 'ext', 100, 'Angry')
distriagrl100,distriagrh100 = Get_Users_topandbottom(PerCorrLH_top500, 'agr', 100, 'Angry')
distrineul100,distrineuh100 = Get_Users_topandbottom(PerCorrLH_top500, 'neu', 100, 'Angry')

print('\n----Angry----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl100, distriopenh100, 10000))
print("Con", mannwhitneyu_bootstrap(districonl100, districonh100, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl100, distriexth100, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl100, distriagrh100, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul100, distrineuh100, 10000))
print('\n--------\n')

distriopenl100,distriopenh100 = Get_Users_topandbottom(PerCorrLH_top500, 'open', 100, 'Sad')
districonl100,districonh100 = Get_Users_topandbottom(PerCorrLH_top500, 'con', 100, 'Sad')
distriextl100,distriexth100 = Get_Users_topandbottom(PerCorrLH_top500, 'ext', 100, 'Sad')
distriagrl100,distriagrh100 = Get_Users_topandbottom(PerCorrLH_top500, 'agr', 100, 'Sad')
distrineul100,distrineuh100 = Get_Users_topandbottom(PerCorrLH_top500, 'neu', 100, 'Sad')

print('\n----Sad----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl100, distriopenh100, 10000))
print("Con", mannwhitneyu_bootstrap(districonl100, districonh100, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl100, distriexth100, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl100, distriagrh100, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul100, distrineuh100, 10000))
print('\n--------\n')

distriopenl100,distriopenh100 = Get_Users_topandbottom(PerCorrLH_top500, 'open', 100, 'Relaxed')
districonl100,districonh100 = Get_Users_topandbottom(PerCorrLH_top500, 'con', 100, 'Relaxed')
distriextl100,distriexth100 = Get_Users_topandbottom(PerCorrLH_top500, 'ext', 100, 'Relaxed')
distriagrl100,distriagrh100 = Get_Users_topandbottom(PerCorrLH_top500, 'agr', 100, 'Relaxed')
distrineul100,distrineuh100 = Get_Users_topandbottom(PerCorrLH_top500, 'neu', 100, 'Relaxed')

print('\n----Relaxed----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl100, distriopenh100, 10000))
print("Con", mannwhitneyu_bootstrap(districonl100, districonh100, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl100, distriexth100, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl100, distriagrh100, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul100, distrineuh100, 10000))
print('\n--------\n')


----Joy----

Open 2177.0, 0.032142645598348694(High, 0.0333)
Con 2367.0, 0.23414381687400765(High, )
Ext 1906.0, 0.004616430170563014(High, 0.0042)
Agr 2545.5, 0.3243410441494383(High, )
Neu 3600.0, 0.02657749907095256(Low, 0.0247)

--------


----Angry----

Open 3922.0, 5.414319317588025e-06(Low, 0.0)
Con 2674.0, 0.9984463432382801(Low, )
Ext 3174.0, 0.029008191716913122(Low, 0.0298)
Agr 3224.5, 0.11760997039285466(Low, )
Neu 2339.0, 0.021029646166532384(High, 0.0201)

--------


----Sad----

Open 2220.0, 0.04796631250856861(High, 0.0486)
Con 3308.0, 0.013472688160339266(Low, 0.0134)
Ext 2878.0, 0.3117684115118118(Low, )
Agr 2450.5, 0.17929080975049838(High, )
Neu 2831.0, 0.5888519966273227(High, )

--------


----Relaxed----

Open 1744.0, 0.00014249581567765194(High, 0.0001)
Con 2614.0, 0.8197823821958325(High, )
Ext 2152.0, 0.06306411900126023(High, )
Agr 2369.5, 0.09942596763300496(High, )
Neu 3533.0, 0.04803640221073277(Low, 0.0484)

--------



# Top 250 and Bottom 250

In [24]:
distriopenl250,distriopenh250 = Get_Users_topandbottom(PerCorrLH_top500, 'open', 250, 'Joy')
districonl250,districonh250 = Get_Users_topandbottom(PerCorrLH_top500, 'con', 250, 'Joy')
distriextl250,distriexth250 = Get_Users_topandbottom(PerCorrLH_top500, 'ext', 250, 'Joy')
distriagrl250,distriagrh250 = Get_Users_topandbottom(PerCorrLH_top500, 'agr', 250, 'Joy')
distrineul250,distrineuh250 = Get_Users_topandbottom(PerCorrLH_top500, 'neu', 250, 'Joy')

print('\n----Joy----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl250, distriopenh250, 10000))
print("Con", mannwhitneyu_bootstrap(districonl250, districonh250, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl250, distriexth250, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl250, distriagrh250, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul250, distrineuh250, 10000))
print('\n--------\n')

distriopenl250,distriopenh250 = Get_Users_topandbottom(PerCorrLH_top500, 'open', 250, 'Angry')
districonl250,districonh250 = Get_Users_topandbottom(PerCorrLH_top500, 'con', 250, 'Angry')
distriextl250,distriexth250 = Get_Users_topandbottom(PerCorrLH_top500, 'ext', 250, 'Angry')
distriagrl250,distriagrh250 = Get_Users_topandbottom(PerCorrLH_top500, 'agr', 250, 'Angry')
distrineul250,distrineuh250 = Get_Users_topandbottom(PerCorrLH_top500, 'neu', 250, 'Angry')

print('\n----Angry----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl250, distriopenh250, 10000))
print("Con", mannwhitneyu_bootstrap(districonl250, districonh250, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl250, distriexth250, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl250, distriagrh250, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul250, distrineuh250, 10000))
print('\n--------\n')

distriopenl250,distriopenh250 = Get_Users_topandbottom(PerCorrLH_top500, 'open', 250, 'Sad')
districonl250,districonh250 = Get_Users_topandbottom(PerCorrLH_top500, 'con', 250, 'Sad')
distriextl250,distriexth250 = Get_Users_topandbottom(PerCorrLH_top500, 'ext', 250, 'Sad')
distriagrl250,distriagrh250 = Get_Users_topandbottom(PerCorrLH_top500, 'agr', 250, 'Sad')
distrineul250,distrineuh250 = Get_Users_topandbottom(PerCorrLH_top500, 'neu', 250, 'Sad')

print('\n----Sad----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl250, distriopenh250, 10000))
print("Con", mannwhitneyu_bootstrap(districonl250, districonh250, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl250, distriexth250, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl250, distriagrh250, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul250, distrineuh250, 10000))
print('\n--------\n')

distriopenl250,distriopenh250 = Get_Users_topandbottom(PerCorrLH_top500, 'open', 250, 'Relaxed')
districonl250,districonh250 = Get_Users_topandbottom(PerCorrLH_top500, 'con', 250, 'Relaxed')
distriextl250,distriexth250 = Get_Users_topandbottom(PerCorrLH_top500, 'ext', 250, 'Relaxed')
distriagrl250,distriagrh250 = Get_Users_topandbottom(PerCorrLH_top500, 'agr', 250, 'Relaxed')
distrineul250,distrineuh250 = Get_Users_topandbottom(PerCorrLH_top500, 'neu', 250, 'Relaxed')

print('\n----Relaxed----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl250, distriopenh250, 10000))
print("Con", mannwhitneyu_bootstrap(districonl250, districonh250, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl250, distriexth250, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl250, distriagrh250, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul250, distrineuh250, 10000))
print('\n--------\n')


----Joy----

Open 13845.0, 0.0032609247345235185(High, 0.0019)
Con 15569.0, 0.07328680830059293(High, )
Ext 13833.0, 0.004115417088949654(High, 0.0036)
Agr 14998.5, 0.0035653407748580646(High, 0.0034)
Neu 20772.0, 0.04741804618365077(Low, 0.0451)

--------


----Angry----

Open 21933.0, 5.261630138919068e-07(Low, 0.0)
Con 18385.0, 0.36732342798107886(Low, )
Ext 19315.0, 0.010835383862290887(Low, 0.0093)
Agr 22091.5, 0.00022708920225198378(Low, 0.0001)
Neu 16247.0, 0.03172985745520163(High, 0.0316)

--------


----Sad----

Open 15017.0, 0.07367504413818633(High, )
Con 20094.0, 0.011207823523773294(Low, 0.0118)
Ext 16912.0, 0.8626820413131133(Low, )
Agr 17270.5, 0.42397946105081163(High, )
Neu 17576.0, 0.35007686504032154(High, )

--------


----Relaxed----

Open 11988.0, 1.8467851368488565e-06(High, 0.0)
Con 16122.0, 0.2069136527522154(High, )
Ext 15013.0, 0.08858208403229885(High, )
Agr 14801.5, 0.001950802559752284(High, 0.002)
Neu 21715.0, 0.004462213412370545(Low, 0.0042)

--------

# Top 500 and Bottom 500

In [25]:
distriopenl500,distriopenh500 = Get_Users_topandbottom(PerCorrLH_top500, 'open', 500, 'Joy')
districonl500,districonh500 = Get_Users_topandbottom(PerCorrLH_top500, 'con', 500, 'Joy')
distriextl500,distriexth500 = Get_Users_topandbottom(PerCorrLH_top500, 'ext', 500, 'Joy')
distriagrl500,distriagrh500 = Get_Users_topandbottom(PerCorrLH_top500, 'agr', 500, 'Joy')
distrineul500,distrineuh500 = Get_Users_topandbottom(PerCorrLH_top500, 'neu', 500, 'Joy')

print('\n----Joy----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl500, distriopenh500, 10000))
print("Con", mannwhitneyu_bootstrap(districonl500, districonh500, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl500, distriexth500, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl500, distriagrh500, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul500, distrineuh500, 10000))
print('\n--------\n')

distriopenl500,distriopenh500 = Get_Users_topandbottom(PerCorrLH_top500, 'open', 500, 'Angry')
districonl500,districonh500 = Get_Users_topandbottom(PerCorrLH_top500, 'con', 500, 'Angry')
distriextl500,distriexth500 = Get_Users_topandbottom(PerCorrLH_top500, 'ext', 500, 'Angry')
distriagrl500,distriagrh500 = Get_Users_topandbottom(PerCorrLH_top500, 'agr', 500, 'Angry')
distrineul500,distrineuh500 = Get_Users_topandbottom(PerCorrLH_top500, 'neu', 500, 'Angry')

print('\n----Angry----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl500, distriopenh500, 10000))
print("Con", mannwhitneyu_bootstrap(districonl500, districonh500, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl500, distriexth500, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl500, distriagrh500, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul500, distrineuh500, 10000))
print('\n--------\n')

distriopenl500,distriopenh500 = Get_Users_topandbottom(PerCorrLH_top500, 'open', 500, 'Sad')
districonl500,districonh500 = Get_Users_topandbottom(PerCorrLH_top500, 'con', 500, 'Sad')
distriextl500,distriexth500 = Get_Users_topandbottom(PerCorrLH_top500, 'ext', 500, 'Sad')
distriagrl500,distriagrh500 = Get_Users_topandbottom(PerCorrLH_top500, 'agr', 500, 'Sad')
distrineul500,distrineuh500 = Get_Users_topandbottom(PerCorrLH_top500, 'neu', 500, 'Sad')

print('\n----Sad----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl500, distriopenh500, 10000))
print("Con", mannwhitneyu_bootstrap(districonl500, districonh500, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl500, distriexth500, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl500, distriagrh500, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul500, distrineuh500, 10000))
print('\n--------\n')

distriopenl500,distriopenh500 = Get_Users_topandbottom(PerCorrLH_top500, 'open', 500, 'Relaxed')
districonl500,districonh500 = Get_Users_topandbottom(PerCorrLH_top500, 'con', 500, 'Relaxed')
distriextl500,distriexth500 = Get_Users_topandbottom(PerCorrLH_top500, 'ext', 500, 'Relaxed')
distriagrl500,distriagrh500 = Get_Users_topandbottom(PerCorrLH_top500, 'agr', 500, 'Relaxed')
distrineul500,distrineuh500 = Get_Users_topandbottom(PerCorrLH_top500, 'neu', 500, 'Relaxed')

print('\n----Relaxed----\n')
print("Open",mannwhitneyu_bootstrap(distriopenl500, distriopenh500, 10000))
print("Con", mannwhitneyu_bootstrap(districonl500, districonh500, 10000))
print("Ext", mannwhitneyu_bootstrap(distriextl500, distriexth500, 10000))
print("Agr",mannwhitneyu_bootstrap(distriagrl500, distriagrh500, 10000))
print("Neu",mannwhitneyu_bootstrap(distrineul500, distrineuh500, 10000))
print('\n--------\n')


----Joy----

Open 61095.0, 0.02544008013665559(High, 0.0258)
Con 63978.0, 0.06181009926157782(High, )
Ext 63361.0, 0.022630296920229536(High, 0.0234)
Agr 63794.5, 0.032593524695260376(High, 0.0331)
Neu 72300.0, 0.2758554555074261(Low, )

--------


----Angry----

Open 81400.0, 1.4351279649543267e-06(Low, 0.0)
Con 72022.0, 0.38688778919106215(Low, )
Ext 77176.0, 0.017000337914133377(Low, 0.0193)
Agr 77993.5, 0.00784735288522839(Low, 0.0073)
Neu 63402.0, 0.05146779491053901(High, )

--------


----Sad----

Open 60985.0, 0.02303378082684021(High, 0.0238)
Con 78229.0, 0.002936085723096148(Low, 0.0035)
Ext 68444.0, 0.573707363375457(High, )
Agr 69860.5, 0.9297516425019574(High, )
Neu 68424.0, 0.8156365171635865(High, )

--------


----Relaxed----

Open 53263.0, 7.198962596808994e-07(High, 0.0)
Con 64605.0, 0.0980041951967101(High, )
Ext 66368.0, 0.20627841051711193(High, )
Agr 60595.5, 0.0012928006498277313(High, 0.0015)
Neu 75396.0, 0.031826594926904636(Low, 0.0314)

--------



## Correlation

In [26]:
def Get_Users_LH_spearman(pers, med, dataset, emotion):
    distril = []
    distrih = [] 
    distrilsper = []
    distrihsper = []
    for i in range(len(dataset)):
        if dataset[i][pers] <= median(med) - th_median and dataset[i]['Ext_song_plays'] >= th_songs\
        and dataset[i]['Ext_song_plays']/dataset[i]['Total_song_plays'] >= th_hitrate:
            distril.append(dataset[i][emotion])
            
        if dataset[i][pers] >= median(med) + th_median and dataset[i]['Ext_song_plays'] >= th_songs \
         and dataset[i]['Ext_song_plays']/dataset[i]['Total_song_plays'] >= th_hitrate:
            distrih.append(dataset[i][emotion])
    
    return distril, distrih, 

In [27]:
def Get_Users_topandbottom_spearman(dataset, pers, num, emotion):
    distril = []
    distrilper = []
    distrih = [] 
    distrihper = []
    temp1 = {}
    temp2 = {}
    for i in dataset:
        temp1[i] = dataset[i][pers]
        temp2[i] = dataset[i][pers]

    temp1 = dict(sorted(temp1.items(), key=lambda item: item[1], reverse=False))
    temp2 = dict(sorted(temp2.items(), key=lambda item: item[1], reverse=True))

    t = 0
    b = 0
    for i in temp1:
        if dataset[i]['Ext_song_plays'] >= th_songs and dataset[i]['Ext_song_plays']/dataset[i]['Total_song_plays'] >= th_hitrate:
            distril.append(dataset[i][emotion])
            distrilper.append(dataset[i][pers])
        t += 1

        if t == num:
            break
    
    for i in temp2:
        if dataset[i]['Ext_song_plays'] >= th_songs and dataset[i]['Ext_song_plays']/dataset[i]['Total_song_plays'] >= th_hitrate:
            distrih.append(dataset[i][emotion])
            distrihper.append(dataset[i][pers])

        b += 1

        if b == num:
            break
    return distril, distrih, distrilper, distrihper


In [28]:
a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Joy'])
        b.append(PerCorrLH[i]['open'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.5f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.5f' % p)
else:
	print('Samples are correlated (reject H0) p=%.5f' % p)
    
a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Angry'])
        b.append(PerCorrLH[i]['open'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.5f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.5f' % p)
else:
	print('Samples are correlated (reject H0) p=%.5f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Sad'])
        b.append(PerCorrLH[i]['open'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.5f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.5f' % p)
else:
	print('Samples are correlated (reject H0) p=%.5f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Relaxed'])
        b.append(PerCorrLH[i]['open'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.5f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.5f' % p)
else:
	print('Samples are correlated (reject H0) p=%.5f' % p)

Spearmans correlation coefficient: 0.08895
Samples are correlated (reject H0) p=0.00289
Spearmans correlation coefficient: -0.17181
Samples are correlated (reject H0) p=0.00000
Spearmans correlation coefficient: 0.08608
Samples are correlated (reject H0) p=0.00394
Spearmans correlation coefficient: 0.18366
Samples are correlated (reject H0) p=0.00000


In [29]:
a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Joy'])
        b.append(PerCorrLH[i]['con'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Angry'])
        b.append(PerCorrLH[i]['con'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Sad'])
        b.append(PerCorrLH[i]['con'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Relaxed'])
        b.append(PerCorrLH[i]['con'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

Spearmans correlation coefficient: 0.056
Samples are uncorrelated (fail to reject H0) p=0.061
Spearmans correlation coefficient: -0.037
Samples are uncorrelated (fail to reject H0) p=0.216
Spearmans correlation coefficient: -0.058
Samples are uncorrelated (fail to reject H0) p=0.054
Spearmans correlation coefficient: 0.041
Samples are uncorrelated (fail to reject H0) p=0.167


In [30]:
a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Joy'])
        b.append(PerCorrLH[i]['ext'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Angry'])
        b.append(PerCorrLH[i]['ext'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Sad'])
        b.append(PerCorrLH[i]['ext'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Relaxed'])
        b.append(PerCorrLH[i]['ext'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

Spearmans correlation coefficient: 0.076
Samples are correlated (reject H0) p=0.011
Spearmans correlation coefficient: -0.086
Samples are correlated (reject H0) p=0.004
Spearmans correlation coefficient: 0.035
Samples are uncorrelated (fail to reject H0) p=0.240
Spearmans correlation coefficient: 0.047
Samples are uncorrelated (fail to reject H0) p=0.112


In [31]:
a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Joy'])
        b.append(PerCorrLH[i]['agr'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Angry'])
        b.append(PerCorrLH[i]['agr'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Sad'])
        b.append(PerCorrLH[i]['agr'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Relaxed'])
        b.append(PerCorrLH[i]['agr'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

Spearmans correlation coefficient: 0.083
Samples are correlated (reject H0) p=0.005
Spearmans correlation coefficient: -0.120
Samples are correlated (reject H0) p=0.000
Spearmans correlation coefficient: 0.076
Samples are correlated (reject H0) p=0.011
Spearmans correlation coefficient: 0.098
Samples are correlated (reject H0) p=0.001


In [32]:
a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Joy'])
        b.append(PerCorrLH[i]['neu'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Angry'])
        b.append(PerCorrLH[i]['neu'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Sad'])
        b.append(PerCorrLH[i]['neu'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

a = []
b = []
for i in PerCorrLH:
    if PerCorrLH[i]['Ext_song_plays'] >= th_songs and PerCorrLH[i]['Ext_song_plays']/PerCorrLH[i]['Total_song_plays'] >= th_hitrate:
        a.append(PerCorrLH[i]['Relaxed'])
        b.append(PerCorrLH[i]['neu'])


coef, p = spearmanr(a, b)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

Spearmans correlation coefficient: -0.045
Samples are uncorrelated (fail to reject H0) p=0.132
Spearmans correlation coefficient: 0.070
Samples are correlated (reject H0) p=0.018
Spearmans correlation coefficient: 0.011
Samples are uncorrelated (fail to reject H0) p=0.713
Spearmans correlation coefficient: -0.095
Samples are correlated (reject H0) p=0.001


# Specific

In [33]:
distrineul100,distrineuh100 = Get_Users_topandbottom(PerCorrLH, 'neu', 100, 'Joy')

In [34]:
mannwhitneyu_bootstrap(distrineul100,distrineuh100,10000)

'3660.0, 0.03850445818935276(Low, 0.0406)'

In [35]:
a, b, c, d = Get_Users_topandbottom_spearman(PerCorrLH, 'neu', 270, 'Angry')

In [36]:
coef, p = spearmanr(a, c)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)
    
coef, p = spearmanr(b, d)
print('Spearmans correlation coefficient: %.3f' % coef)

alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

Spearmans correlation coefficient: -0.013
Samples are uncorrelated (fail to reject H0) p=0.847
Spearmans correlation coefficient: 0.074
Samples are uncorrelated (fail to reject H0) p=0.280
